In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install langchain
!pip install unstructured
!pip install pytesseract
!pip install sentence_transformers
!pip install llama_index
!pip install faiss-cpu
!pip install pypdf
!pip install xformers

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import  LangchainEmbedding
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from langchain.text_splitter import TokenTextSplitter
import torch
import transformers







In [ ]:

################## create vector store from pdf
loader = PyPDFLoader("Sheraton.pdf")
pages = loader.load_and_split()
raw_text = ""
for p in pages:
    raw_text += p.page_content

text_splitter = TokenTextSplitter(chunk_size=150, chunk_overlap=0)
chunks=text_splitter.split_text(raw_text)
hfemb = HuggingFaceEmbeddings()
models_docs = [Document(page_content=chunks[i], metadata={"index": i}) for i in range( len(chunks))]
models_vector_store = FAISS.from_documents(models_docs,hfemb)

In [ ]:
#load LLM
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipeline = transformers.pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=250,
        do_sample=True,
        top_k=10,
        temperature=0.001,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [ ]:
#############
if __name__=="__main__":

    qusetion="How can I access the internet in my room?"

    models_retriever = models_vector_store.as_retriever()
    docs = models_retriever.get_relevant_documents(qusetion)
    context = docs[0].page_content.replace('\n',' ')

    prompt=f"using following context to answer the question\nquestion:{qusetion}\ncontext:{context}\n\nanswer: "
    sequences = pipeline(prompt)
    sequences[0]['generated_text'].split('\n\nanswer')[1]